# Prepare dataset and model

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [ ]:
# load dataset

input_size=32
num_workers = 2

transform_train = transforms.Compose(
     [transforms.RandomCrop(32, 4),
    transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])])

transform_val = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
])

batch_size = 64

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers, pin_memory=True)

valset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_val)
valloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers, pin_memory=True)

classes = ('apples', 'aquarium fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottles',
           'bowls', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'cans', 'castle', 'caterpillar', 'cattle', 'chair',
           'chimpanzee', 'clock', 'cloud', 'cockroach', 'computer keyboard', 'couch', 'crab', 'crocodile',
           'cups', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house',
           'kangaroo', 'lamp', 'lawn-mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple', 'motorcycle',
           'mountain', 'mouse', 'mushrooms', 'oak', 'oranges', 'orchids', 'otter', 'palm', 'pears', 'pickup truck',
           'pine', 'plain', 'plates', 'poppies', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket',
           'roses', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel',
           'streetcar', 'sunflowers', 'sweet peppers', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor',
           'train', 'trout', 'tulips', 'turtle', 'wardrobe', 'whale', 'willow', 'wolf', 'woman', 'worm')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# define ResNet model

import torch.nn as nn
import torch.nn.functional as F
from typing import Optional



class BasicBlock(nn.Module):
    """Basic Residual Block."""

    def __init__(
        self, in_channels: int, out_channels: int,
        stride: Optional[int] = 1, downsample: Optional[nn.Module] = None
    ) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=out_channels)

        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward Pass."""
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        return out


class ResNet(nn.Module):
    """Residual Neural Network."""

    def __init__(self, num_classes: Optional[int] = 100, in_channels: Optional[int] = 32) -> None:
        super().__init__()

        # Pre
        self.in_channels = in_channels
        self.conv1 = nn.Conv2d(3, 32, 3, 1, 1, bias=False)
        self.bn = nn.BatchNorm2d(num_features=32)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout2d(p=0.02)

        # Basic Blocks
        self.conv2_x = nn.Sequential(*[
            BasicBlock(32, 32) for i in range(4)
        ])

        self.conv3_x = nn.Sequential(BasicBlock(
            32, 64, 2,
            nn.Sequential(nn.Conv2d(32, 64, 3, 2, 1, bias=False), nn.BatchNorm2d(64))
        ))

        self.conv4_x = nn.Sequential(BasicBlock(
            64, 128, 2,
            nn.Sequential(nn.Conv2d(64, 128, 3, 2, 1, bias=False), nn.BatchNorm2d(128))
        ))

        self.conv5_x = nn.Sequential(BasicBlock(
            128, 256, 2,
            nn.Sequential(nn.Conv2d(128, 256, 3, 2, 1, bias=False), nn.BatchNorm2d(256))
        ))

        # Post
        self.maxpool = nn.MaxPool2d(kernel_size=4, stride=1)
        self.fc_layer = nn.Linear(256, num_classes)

        # initialize weights

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data, mode='fan_out')
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass."""
        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)

        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)

        return x


# Training

In [ ]:
# initialize model

net = ResNet()
net.to(device)

ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (dropout): Dropout2d(p=0.02, inplace=False)
  (block32): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_size=(3

In [ ]:
# initialize loss function and optimizer

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:

num_epoch = 50
PATH = './resnet-pro.pth'
min_val_loss = np.inf

# training loop
for epoch in range(num_epoch):

    # training step
    running_train_loss = torch.tensor(0.0, device=device)
    running_train_acc = torch.tensor(0.0, device=device)
    net.train()

    for i, data in enumerate(trainloader, 0):

        inputs = data[0].to(device, non_blocking=True)
        labels = data[1].to(device, non_blocking=True)

        optimizer.zero_grad()

        outputs = net(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_train_loss = torch.add(running_train_loss, loss, alpha=inputs.size(0))
        running_train_acc = torch.add(
                    running_train_acc, torch.sum(preds == labels.data), alpha=1.0/labels.size(0)
        )

    epoch_train_loss: float = running_train_loss.cpu().detach().numpy() / len(trainloader) / batch_size
    epoch_train_acc: float = running_train_acc.cpu().detach().numpy() / len(trainloader)

    print(f"Epoch {epoch+1} \t\t Training Loss: {epoch_train_loss} Training accuracy: {epoch_train_acc*100:.2f} %")

    # validation step
    running_val_loss = torch.tensor(0.0, device=device)
    running_val_acc = torch.tensor(0.0, device=device)

    with torch.no_grad():

        net.eval()
        for data in valloader:
            inputs = data[0].to(device, non_blocking=True)
            labels = data[1].to(device, non_blocking=True)

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            _, pred = torch.max(outputs, dim=1)
            running_val_loss = torch.add(
                running_val_loss, loss, alpha=inputs.size(0)
            )
            running_val_acc = torch.add(
                running_val_acc, torch.sum(pred == labels.data), alpha= 1.0/labels.size(0)
            )

    epoch_val_loss: float = running_val_loss.cpu().detach().numpy() / len(testloader) / batch_size
    epoch_val_acc: float = running_val_acc.cpu().detach().numpy() / len(testloader)

    print(
        f"Validation Loss: {epoch_val_loss} Validation Accuracy: {epoch_val_acc*100:.2f} %"
    )

    # save model weights if validation loss decreased
    if min_val_loss > epoch_val_loss:
        print(
            f"Validation loss decreased({min_val_loss:.6f}--->\
            {epoch_val_loss:.6f})"
        )
        min_val_loss = epoch_val_loss
        print(f"Saving model state dict to {PATH}")
        torch.save(net.state_dict(), PATH)

print('Finished Training')